# Save small local files to forces
<div class="alert alert-danger alert-info">
    <b>It is important to save your results in a place that can last longer than a few days/weeks!</b>
</div>
- When you have saved data locally on your JupyterLab instance and you want to mak a backup on https://forces2021.uiogeo-apps.sigma2.no/

In [ ]:
import os
import pathlib
import s3fs
import xarray as xr

## Get a sample file

In [ ]:
ds = xr.tutorial.open_dataset("air_temperature.nc").rename({"air": "Tair"})

## Save sample file into local file 
- The file is small (< 5GB so it is not an issue to do that)

In [ ]:
ds.load().to_netcdf("Tair_temperature.nc")

## Save your results to Remote private object storage
- your credentials are in `$HOME/.aws/credentials` 
- check with your instructor to get the secret access key (replace XXX by the right key)

```
[default]
aws_access_key_id=forces2021-work
aws_secret_access_key=XXXXXXXXXXXX
aws_endpoint_url=https://forces2021.uiogeo-apps.sigma2.no/
```

In [ ]:
target = s3fs.S3FileSystem(
    anon=False,
    client_kwargs={"endpoint_url": "https://forces2021.uiogeo-apps.sigma2.no/"},
)

In [ ]:
s3_prefix = "s3://work/annefou/test/"
print(s3_prefix)

In [ ]:
import glob

list_files = glob.glob("*.nc")

In [ ]:
list_files

In [ ]:
for file in list_files:
    s3_path_file = os.path.join(s3_prefix, os.path.basename(file))
    print(file, s3_path_file)
    target.put(file, s3_path_file)

## Then you can use the remote file

In [ ]:
remote_file = ["work/annefou/test/Tair_temperature.nc"]

In [ ]:
fileset = [target.open(file) for file in remote_file]

In [ ]:
ds_check = xr.open_mfdataset(fileset, combine="by_coords")
ds_check